In [3]:
#Import Necessary Libraries
import pandas as pd
import numpy as np

# Student Class

In [4]:
class Student:
    def __init__(self, name, student_id):
        """
        Initialize a new student with a name and student ID.
        """
        self.name = name
        self.student_id = student_id
        self.courses = []
        self.grades = {}

    def add_course(self, course_code, grade):
        """
        Enroll the student in a course and assign a grade.
        """
        self.courses.append(course_code)
        self.grades[course_code] = grade


# Course Class

In [5]:
class Course:
    def __init__(self, code, name, credits, capacity):
        """
        Initialize a new course with a code, name, credit hours, and capacity.
        """
        self.code = code
        self.name = name
        self.credits = credits
        self.capacity = capacity
        self.enrolled_students = []

    def enroll_student(self, student_id):
        """
        Attempt to enroll a student. Return True if successful, False if the course is full.
        """
        if len(self.enrolled_students) < self.capacity:
            self.enrolled_students.append(student_id)
            return True
        else:
            return False


# Course Managment System class

In [11]:
import pandas as pd
import numpy as np

class CourseManagementSystem:
    def __init__(self):
        """
        Initialize the CourseManagementSystem with DataFrames for students, courses, and a waitlist.
        """
        self.students = pd.DataFrame(columns=['Name', 'StudentID', 'Grades'])
        self.courses = pd.DataFrame(columns=['Code', 'Name', 'Credits', 'Capacity', 'EnrolledStudents'])
        self.waitlist = pd.DataFrame(columns=['StudentID', 'CourseCode'])

    def add_student(self, name, student_id):
        """
        Add a new student to the system.
        """
        new_student = pd.DataFrame([{'Name': name, 'StudentID': student_id, 'Grades': {}}])
        self.students = pd.concat([self.students, new_student], ignore_index=True)

    def add_course(self, code, name, credits, capacity):
        """
        Add a new course to the system.
        """
        new_course = pd.DataFrame([{'Code': code, 'Name': name, 'Credits': credits, 'Capacity': capacity, 'EnrolledStudents': []}])
        self.courses = pd.concat([self.courses, new_course], ignore_index=True)

    def enroll_student(self, student_id, course_code):
        """
        Enroll a student in a course, or add them to the waitlist if the course is full.
        """
        course_index = self.courses[self.courses['Code'] == course_code].index[0]
        if len(self.courses.at[course_index, 'EnrolledStudents']) < self.courses.at[course_index, 'Capacity']:
            self.courses.at[course_index, 'EnrolledStudents'].append(student_id)
        else:
            new_waitlist_entry = pd.DataFrame([{'StudentID': student_id, 'CourseCode': course_code}])
            self.waitlist = pd.concat([self.waitlist, new_waitlist_entry], ignore_index=True)

    def calculate_total_credits(self, student_id):
        """
        Calculate the total number of credits a student is enrolled in.
        """
        student_courses = self.courses[self.courses['EnrolledStudents'].apply(lambda x: student_id in x)]
        return student_courses['Credits'].sum()

    def find_students_in_course(self, course_code):
        """
        Find all students enrolled in a specific course.
        """
        course = self.courses.loc[self.courses['Code'] == course_code].iloc[0]
        return course['EnrolledStudents']

    def get_most_popular_course(self):
        """
        Find the most popular course based on enrollment numbers.
        """
        self.courses['NumEnrolled'] = self.courses['EnrolledStudents'].apply(len)
        most_popular_course = self.courses.loc[self.courses['NumEnrolled'].idxmax()]
        return most_popular_course['Code']

    def remove_course(self, course_code):
        """
        Remove a course from the system.
        """
        self.courses = self.courses[self.courses['Code'] != course_code]

    def calculate_gpa(self, student_id):
        """
        Calculate the GPA of a student based on a 4.0 scale.
        """
        student = self.students.loc[self.students['StudentID'] == student_id].iloc[0]
        grades = student['Grades']
        if grades:
            # Map the grades to a 4.0 scale
            def map_to_gpa(grade):
                if grade >= 93:
                    return 4.0
                elif grade >= 85:
                    return 3.7
                elif grade >= 79:
                    return 3.3
                elif grade >= 75:
                    return 3.0
                elif grade >= 71:
                    return 2.7
                elif grade >= 68:
                    return 2.3
                elif grade >= 65:
                    return 2.0
                elif grade >= 60:
                    return 1.7
                elif grade >= 55:
                    return 1.3
                elif grade >= 50:
                    return 1.0
                else:
                    return 0.0
            gpa_list = [map_to_gpa(grade) for grade in grades.values()]
            gpa = np.mean(gpa_list)
        else:
            gpa = 0.0
        return round(gpa,2)

    def get_student_courses(self, student_id):
        """
        Get a list of courses a student is enrolled in.
        """
        student = self.students.loc[self.students['StudentID'] == student_id].iloc[0]
        return list(student['Grades'].keys())

    def add_to_waitlist(self, student_id, course_code):
        """
        Add a student to the waitlist for a course.
        """
        new_waitlist_entry = pd.DataFrame([{'StudentID': student_id, 'CourseCode': course_code}])
        self.waitlist = pd.concat([self.waitlist, new_waitlist_entry], ignore_index=True)


# I/O Ops and logging

In [8]:
def load_students(filename):
    """
    Load student data from a CSV file.
    """
    return pd.read_csv(filename)

def load_courses(filename):
    """
    Load course data from a CSV file.
    """
    return pd.read_csv(filename)

def save_students(students, filename):
    """
    Save student data to a CSV file.
    """
    students.to_csv(filename, index=False)

def save_courses(courses, filename):
    """
    Save course data to a CSV file.
    """
    courses.to_csv(filename, index=False)

def log_operation(message):
    """
    Log an operation to a text file.
    """
    with open('operations.log', 'a') as f:
        f.write(f'{message}\n')


# Main function

In [9]:
def main():
    cms = CourseManagementSystem()
    # Load initial data
    try:
        cms.students = load_students('students.csv')
        cms.courses = load_courses('courses.csv')
    except FileNotFoundError as e:
        log_operation(f'File error: {e}')
    
    try:
        cms.add_student("John Doe", 123)
        cms.add_course("CS101", "Computer Science", 3, 30)
        cms.enroll_student(123, "CS101")
        gpa = cms.calculate_gpa(123)
        print(f"GPA: {gpa}")
        save_students(cms.students, 'students.csv')
        save_courses(cms.courses, 'courses.csv')
    except Exception as e:
        log_operation(f'Error: {e}')

if __name__ == "__main__":
    main()


# Test Case

In [15]:
def test_course_management_system():
    cms = CourseManagementSystem()

    # Test adding students
    cms.add_student("Alice Johnson", 1001)
    cms.add_student("Bob Smith", 1002)
    cms.add_student("Charlie Brown", 1003)
    print("\n--- Students Added ---")
    print(cms.students)

    # Test adding courses
    cms.add_course("MATH101", "Calculus I", 4, 2)
    cms.add_course("CS102", "Introduction to Programming", 3, 3)
    cms.add_course("HIST202", "World History", 3, 2)
    print("\n--- Courses Added ---")
    print(cms.courses)

    # Test enrolling students
    cms.enroll_student(1001, "MATH101")
    cms.enroll_student(1002, "MATH101")  # This should fill up the course
    cms.enroll_student(1003, "MATH101")  # This student should go on the waitlist
    cms.enroll_student(1001, "CS102")
    cms.enroll_student(1002, "CS102")
    cms.enroll_student(1003, "CS102")
    cms.enroll_student(1001, "HIST202")
    print("\n--- Students Enrolled ---")
    print(cms.courses)
    print("\n--- Waitlist ---")
    print(cms.waitlist)

    # Test calculating total credits
    print("\n--- Total Credits ---")
    print(f"Alice's Total Credits: {cms.calculate_total_credits(1001)}")
    print(f"Bob's Total Credits: {cms.calculate_total_credits(1002)}")
    print(f"Charlie's Total Credits: {cms.calculate_total_credits(1003)}")

    # Assign grades to students
    cms.students.at[0, 'Grades'] = {'MATH101': 95, 'CS102': 85, 'HIST202': 88}
    cms.students.at[1, 'Grades'] = {'MATH101': 88, 'CS102': 79}
    cms.students.at[2, 'Grades'] = {'CS102': 92}

    # Test calculating GPA
    print("\n--- GPA Calculation ---")
    print(f"Alice's GPA: {cms.calculate_gpa(1001):.2f}")
    print(f"Bob's GPA: {cms.calculate_gpa(1002):.2f}")
    print(f"Charlie's GPA: {cms.calculate_gpa(1003):.2f}")

    # Test finding students in a course
    print("\n--- Students in Courses ---")
    print(f"Students in MATH101: {cms.find_students_in_course('MATH101')}")
    print(f"Students in CS102: {cms.find_students_in_course('CS102')}")
    print(f"Students in HIST202: {cms.find_students_in_course('HIST202')}")

    # Test most popular course
    print("\n--- Most Popular Course ---")
    print(f"Most Popular Course: {cms.get_most_popular_course()}")

    # Test removing a course
    cms.remove_course("HIST202")
    print("\n--- Courses after Removing HIST202 ---")
    print(cms.courses)

    # Test getting student courses
    print("\n--- Student Courses ---")
    print(f"Alice's Courses: {cms.get_student_courses(1001)}")
    print(f"Bob's Courses: {cms.get_student_courses(1002)}")
    print(f"Charlie's Courses: {cms.get_student_courses(1003)}")

    # Test adding to waitlist
    cms.add_to_waitlist(1001, "MATH101")
    print("\n--- Waitlist after Adding Alice ---")
    print(cms.waitlist)

if __name__ == "__main__":
    test_course_management_system()



--- Students Added ---
            Name StudentID Grades
0  Alice Johnson      1001     {}
1      Bob Smith      1002     {}
2  Charlie Brown      1003     {}

--- Courses Added ---
      Code                         Name Credits Capacity EnrolledStudents
0  MATH101                   Calculus I       4        2               []
1    CS102  Introduction to Programming       3        3               []
2  HIST202                World History       3        2               []

--- Students Enrolled ---
      Code                         Name Credits Capacity    EnrolledStudents
0  MATH101                   Calculus I       4        2        [1001, 1002]
1    CS102  Introduction to Programming       3        3  [1001, 1002, 1003]
2  HIST202                World History       3        2              [1001]

--- Waitlist ---
  StudentID CourseCode
0      1003    MATH101

--- Total Credits ---
Alice's Total Credits: 10
Bob's Total Credits: 7
Charlie's Total Credits: 3

--- GPA Calculation --